In [1]:
import fitz  # PyMuPDF
import pprint # Para imprimir bonito
from pathlib import Path# Tu Endpoint (ya incluye https:// y el puerto 443 está implícito)

In [2]:
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

c:\Users\fguerrero\Documents\Especializaciones\URP\Inteligencia Artificial Generativa\Trabajos\Proyecto I\CFAAgent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PDF_PATH = Path("data/cfa_books/V1 - Quantitive Methods (CFA-L1-2025).pdf")
NEW_INDEX_NAME = "cfa_documents_structured" # ¡Usaremos un índice nuevo!

# Configuración del "chunker"
CHUNK_SIZE = 1200
CHUNK_OVERLAP = 250

In [ ]:
print("✅ Librerías y constantes cargadas.")
print(f"Libro a procesar: {PDF_PATH.name}")
print(f"Índice de destino: {NEW_INDEX_NAME}")

✅ Librerías y constantes cargadas.
Libro a procesar: V1 - Quantitive Methods (CFA-L1-2025).pdf
Índice de destino: cfa_documents_structured


In [ ]:
# Célula 2: Extraer el Índice (ToC)
import fitz
import re

print(f"Abriendo: {PDF_PATH.name} para extraer el índice...")
doc = fitz.open(PDF_PATH)

# --- Opción A: Extracción Automática (Si el PDF tiene marcadores) ---
toc = doc.get_toc()

if not toc:
    print("❌ PDF no tiene ToC automático. Usando Plan B (RegEx en primeras páginas)...")
    
    # --- Opción B: Extracción Manual (Plan B) ---
    # Leer el texto de las primeras páginas (ej. 3 a 10) donde está el índice.
    toc_text = ""
    for page_num in range(2, 10): # Ajusta este rango según tu PDF
        toc_text += doc.load_page(page_num).get_text()
    
    # Expresión Regular para encontrar líneas del índice (ej. "Reading 1: ... 11")
    # Esto puede necesitar ajuste fino
    pattern = re.compile(r'^(Reading \d+:[^.\n]+)\.{5,}\s*(\d+)$', re.MULTILINE)
    
    toc = []
    for match in pattern.finditer(toc_text):
        title = match.group(1).strip()
        page = int(match.group(2))
        toc.append([1, title, page]) # [Nivel 1, Título, Página]
        
if not toc:
    print("❌ ERROR: No se pudo extraer el índice. No se pueden crear metadatos.")
else:
    # --- Limpiar y Crear el "Mapa de Secciones" ---
    section_map = []
    for level, title, page in toc:
        # Solo nos interesan los capítulos principales (Nivel 1)
        if level == 1:
            section_map.append({
                "topic": title,
                "start_page": page
            })
    
    print("\n--- 🗺️ Mapa de Secciones Creado ---")
    pprint.pprint(section_map)
    print("-----------------------------------")